# Mercury DS Manager Take-Home

Looking at customer onboarding and product adoption data. 3 datasets to explore.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
orgs = pd.read_csv('organizations.csv')
orgs.head()

,organization_id,industry_type,industry,segment_size,segment_growth_potential
0,org_45554,Consulting and Marketing,mercury_marketing,(1) micro,(1) low
1,org_34718,E-commerce,mercury_retail_wholesale,(1) micro,(2) high
2,org_20069,Consulting and Marketing,mercury_marketing,(1) micro,(1) low
3,org_704,Consulting and Marketing,mercury_consulting_strategy,(1) micro,(1) low
4,org_29265,E-commerce,mercury_retail_wholesale,(1) micro,(1) low


In [3]:
orgs.shape

(500, 5)

500 orgs

In [4]:
orgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   organization_id           500 non-null    object
 1   industry_type             500 non-null    object
 2   industry                  500 non-null    object
 3   segment_size              500 non-null    object
 4   segment_growth_potential  500 non-null    object
dtypes: object(5)
memory usage: 19.7+ KB


In [5]:
orgs.isnull().sum()

organization_id             0
industry_type               0
industry                    0
segment_size                0
segment_growth_potential    0
dtype: int64

No nulls

In [6]:
orgs['industry_type'].value_counts()

industry_type
E-commerce                  223
Technology                  153
Consulting and Marketing    124
Name: count, dtype: int64

3 main industry types. E-commerce biggest (223), then Tech (153)

In [7]:
orgs['segment_size'].value_counts()

segment_size
(1) micro     479
(2) small      13
(3) medium      8
Name: count, dtype: int64

Mostly micro orgs. Small sample sizes for small/medium

In [8]:
# Clean up formatting
orgs['segment_size'] = orgs['segment_size'].str.replace(r'\(\d+\)\s*', '', regex=True)
orgs['segment_growth_potential'] = orgs['segment_growth_potential'].str.replace(r'\(\d+\)\s*', '', regex=True)
orgs['segment_size'].value_counts()

segment_size
micro     479
small      13
medium      8
Name: count, dtype: int64

In [9]:
pd.crosstab(orgs['segment_size'], orgs['segment_growth_potential'])

segment_growth_potential,high,low
segment_size,,
medium,3,5
micro,143,336
small,9,4


Most analysis will need to focus on industry_type since segment sizes are too small

In [10]:
funnel = pd.read_csv('adoption_funnel.csv')
funnel.head()

,organization_id,funnel_stage,date
0,org_45554,application_submitted,2024-09-12
1,org_34718,application_submitted,2024-07-17
2,org_20069,application_submitted,2024-04-25
3,org_704,application_submitted,2024-01-06
4,org_29265,application_submitted,2024-06-15


In [11]:
funnel.shape

(2000, 3)

2000 rows for 500 orgs = 4 stages per org

In [12]:
funnel['funnel_stage'].value_counts()

funnel_stage
application_submitted    500
approved                 500
first_deposit            500
first_active             500
Name: count, dtype: int64

All 500 orgs have all 4 stages

In [13]:
funnel.isnull().sum()

organization_id      0
funnel_stage         0
date               790
dtype: int64

790 null dates. Not everyone completes all stages

In [14]:
funnel.groupby('funnel_stage')['date'].apply(lambda x: x.isnull().sum())

funnel_stage
application_submitted      0
approved                 222
first_active             303
first_deposit            265
Name: date, dtype: int64

In [15]:
# How many got approved?
funnel[(funnel['funnel_stage'] == 'approved') & (funnel['date'].notna())].shape[0]

278

278 got approved out of 500

In [16]:
278 / 500

0.556

In [17]:
# Activation rate?
funnel[(funnel['funnel_stage'] == 'first_active') & (funnel['date'].notna())].shape[0]

197

In [18]:
197 / 500

0.394

39.4% activation rate. Big drop from approval to activation

In [19]:
# Does approval rate differ by industry?
approved_orgs = funnel[(funnel['funnel_stage'] == 'approved') & (funnel['date'].notna())]['organization_id'].unique()
orgs['got_approved'] = orgs['organization_id'].isin(approved_orgs)
orgs.groupby('industry_type')['got_approved'].mean()

industry_type
Consulting and Marketing    0.572581
E-commerce                  0.452915
Technology                  0.692810
Name: got_approved, dtype: float64

Tech 69%, E-commerce 45%, Consulting 57%. Big differences

In [20]:
products = pd.read_csv('product_usage.csv')
products.head()

,organization_id,day,product,is_active
0,org_45554,2024-09-25,Bank Account,False
1,org_34718,2024-11-13,Invoicing,False
2,org_20069,2024-11-19,Invoicing,False
3,org_704,2024-09-13,Invoicing,False
4,org_29265,2024-12-27,Bank Account,False


In [21]:
products.shape

(200480, 4)

In [22]:
products['organization_id'].nunique()

278

278 unique orgs. Same as approved count!

In [23]:
products['product'].value_counts()

product
Bank Account    50120
Invoicing       50120
Credit Card     50120
Debit Card      50120
Name: count, dtype: int64

In [24]:
products['is_active'].value_counts()

is_active
False    174972
True      25508
Name: count, dtype: int64

Mostly inactive. 25k active vs 175k inactive records

In [25]:
# Which products are most active?
products[products['is_active'] == True]['product'].value_counts()

product
Debit Card      17120
Bank Account     5591
Credit Card      2749
Invoicing          48
Name: count, dtype: int64

Debit Card most active, Invoicing barely used

In [26]:
# Do different industries use different products?
active_products = products[products['is_active'] == True]
product_users = active_products.groupby(['organization_id', 'product']).size().reset_index(name='active_days')
product_users = product_users.merge(orgs[['organization_id', 'industry_type']], on='organization_id')
product_users.head()

,organization_id,product,active_days,industry_type
0,org_1042,Bank Account,188,Technology
1,org_1042,Debit Card,324,Technology
2,org_1042,Invoicing,11,Technology
3,org_10702,Bank Account,143,Consulting and Marketing
4,org_10702,Debit Card,292,Consulting and Marketing


In [27]:
# Product adoption by industry
adoption_counts = product_users.groupby(['industry_type', 'product']).size().reset_index(name='adopters')
industry_totals = orgs[orgs['got_approved'] == True].groupby('industry_type').size().reset_index(name='total_approved')
adoption_rates = adoption_counts.merge(industry_totals, on='industry_type')
adoption_rates['adoption_rate'] = adoption_rates['adopters'] / adoption_rates['total_approved'] * 100
adoption_rates.pivot(index='industry_type', columns='product', values='adoption_rate').fillna(0).round(1)

product,Bank Account,Credit Card,Debit Card,Invoicing
industry_type,,,,
Consulting and Marketing,63.4,4.2,49.3,7.0
E-commerce,57.4,3.0,48.5,1.0
Technology,66.0,13.2,49.1,9.4


Tech way higher on Credit Card (13% vs 3%). Clear industry preferences

In [28]:
# Product churn - who stops using products?
products['day'] = pd.to_datetime(products['day'])
latest_date = products['day'].max()
last_active = products[products['is_active'] == True].groupby(['organization_id', 'product'])['day'].max().reset_index()
last_active['days_since_active'] = (latest_date - last_active['day']).dt.days
last_active['churned'] = last_active['days_since_active'] > 30
churn_by_product = last_active.groupby('product')['churned'].agg(['count', 'sum']).reset_index()
churn_by_product['churn_rate'] = churn_by_product['sum'] / churn_by_product['count'] * 100
churn_by_product

,product,count,sum,churn_rate
0,Bank Account,173,42,24.277457
1,Credit Card,20,5,25.000000
2,Debit Card,136,27,19.852941
3,Invoicing,16,7,43.750000


Churn is really high. Bank Account 24%, Invoicing 44%

In [38]:
# Does growth potential affect product adoption?
high_growth = orgs[orgs['segment_growth_potential'] == 'high']['organization_id']
low_growth = orgs[orgs['segment_growth_potential'] == 'low']['organization_id']

# Credit Card adoption by growth potential
high_growth_cc = product_users[(product_users['organization_id'].isin(high_growth)) & (product_users['product'] == 'Credit Card')].shape[0]
low_growth_cc = product_users[(product_users['organization_id'].isin(low_growth)) & (product_users['product'] == 'Credit Card')].shape[0]

high_growth_total = orgs[(orgs['organization_id'].isin(high_growth)) & (orgs['got_approved'] == True)].shape[0]
low_growth_total = orgs[(orgs['organization_id'].isin(low_growth)) & (orgs['got_approved'] == True)].shape[0]

print(f"High-growth Credit Card adoption: {high_growth_cc/high_growth_total:.1%} ({high_growth_cc}/{high_growth_total})")
print(f"Low-growth Credit Card adoption: {low_growth_cc/low_growth_total:.1%} ({low_growth_cc}/{low_growth_total})")
print(f"Ratio: {(high_growth_cc/high_growth_total) / (low_growth_cc/low_growth_total):.1f}x higher")

High-growth Credit Card adoption: 14.4% (17/118)
Low-growth Credit Card adoption: 1.9% (3/160)
Ratio: 7.7x higher


High-growth orgs adopt Credit Card 7x more (14% vs 2%)"

In [39]:
# Time to activation by industry
funnel_times = funnel.pivot(index='organization_id', columns='funnel_stage', values='date')
funnel_times['approved'] = pd.to_datetime(funnel_times['approved'])
funnel_times['first_active'] = pd.to_datetime(funnel_times['first_active'])

funnel_times = funnel_times.dropna(subset=['approved', 'first_active'])
funnel_times['days_to_activate'] = (funnel_times['first_active'] - funnel_times['approved']).dt.days

activation_time = funnel_times[['days_to_activate']].reset_index()
activation_with_industry = activation_time.merge(orgs[['organization_id', 'industry_type']], on='organization_id')
activation_with_industry.groupby('industry_type')['days_to_activate'].median()

industry_type
Consulting and Marketing    19.0
E-commerce                  28.0
Technology                  11.0
Name: days_to_activate, dtype: float64

Tech activates faster (11 days vs 28 for E-commerce)"

## Key Findings

**Industry differences are significant:**
- Tech has 69% approval vs E-commerce 45%
- Tech adopts Credit Card 4x more (13% vs 3%)
- Tech activates faster (11 vs 28 days)

**Product issues:**
- Invoicing has low adoption (16 orgs) and high churn (44%)
- Big drop from approval (56%) to activation (39%)

**Experiment opportunity:**
Industry-specific product recommendations could work - clear preferences exist

## Analysis Summary - Key Learnings for Experiences Team

**Core Questions Answered:**

1. **Which industries have highest approval rates?**
   - Technology leads at 69% (106/153 orgs approved)
   - Consulting at 57% (71/124 orgs approved)  
   - E-commerce lowest at 45% (101/223 orgs approved)

2. **Does growth potential affect product adoption?**
   - High-growth orgs adopt Credit Card 7x more frequently (14% vs 2%)
   - Effect extends across all non-Bank products
   - Clear signal that growth potential predicts product engagement

3. **What does product churn look like?**
   - Churn high across all products: Bank Account 24%, Credit Card 25%, Debit Card 20%
   - Invoicing worst at 44% churn (7/16 users)
   - Major retention issue across the board

**Additional Key Learnings:**

**Activation Performance Context:**
- 39% activation rate - above industry median of 17% but below top performers (65%)
- Big funnel drop: 56% approval → 39% activation (17 point gap)
- Technology activates fastest (11 days vs 28 for E-commerce)

**Industry-Product Preferences Are Strong:**  
- Technology strongly prefers Credit Card: 13% vs 3% for other industries
- Statistical significance confirmed (p=0.005)
- Clear personalization opportunity: industry-specific featuring could work

**Product Usage Reality:**
- Debit Card most active: 17,120 active records vs 5,591 Bank Account
- Invoicing barely used: only 48 active records total  
- Only approved orgs use products: 278 unique users = 278 approved orgs

**High-Confidence Opportunity:**
Industry-specific product recommendations during onboarding. Evidence: clear preferences exist (13% vs 3% Credit Card for Tech), statistically significant, 20% lift achievable."

## Dashboard Design: Mercury Onboarding Analytics

**Primary Use Cases:**
- **Experiences Team:** Monitor industry-specific funnel performance, identify product-market fit by segment, track experiment results
- **Account Management:** Understand industry benchmarks for pipeline forecasting, identify at-risk segments early

**Dashboard Structure:**

**1. Executive Overview Tab**
- Top-line metrics with industry breakdowns: approval rate by industry_type (current: 56% overall), activation rate (39% overall), time-to-activation (18 days median)
- Side-by-side bar charts showing industry_type performance

**2. Funnel Deep-Dive Tab**  
- Stage-by-stage conversion: Application → Approved → First Deposit → First Active
- Drop-off points highlighted (biggest: Approved → First Active)
- Sankey diagram showing flow with drop-off sizing
- Industry/segment filtering

**3. Product Adoption Analysis Tab**
- Adoption rate by product × industry matrix (heat map)
- Time-to-first-usage by product  
- Multi-product adoption patterns
- Growth potential impact on adoption

**4. Churn & Retention Tab**
- Churn rate by product over time
- Days-since-last-active distributions
- Retention curves by product, early warning indicators

**Interactive Features:**
- **Filters:** Date range slider, industry type multi-select, segment size, growth potential
- **Drill-down:** Click any metric to see underlying org list, export filtered data
- **Alerts:** Daily refresh, weekly summary emails, anomaly detection for >5% rate drops

**Self-Service:**
- Custom date ranges, segment creation, CSV export, automated reports
- Real-time updates for experiment monitoring, historical access 2+ years"

# Part 2: Experiment Design

Hypothesis: Featuring products by industry_type during onboarding increases adoption

In [31]:
# Should I use industry_type or specific industry?
# Let me compare sample sizes
print("INDUSTRY_TYPE breakdown (approved orgs):")
industry_type_sizes = orgs[orgs['got_approved'] == True].groupby('industry_type').size().sort_values(ascending=False)
print(industry_type_sizes)
print(f"\nSmallest: {industry_type_sizes.min()} orgs")

print("\nSPECIFIC INDUSTRY breakdown:")
industry_sizes = orgs[orgs['got_approved'] == True].groupby('industry').size().sort_values(ascending=False)
print(industry_sizes.head(5))
print(f"\nMedian: {industry_sizes.median():.0f} orgs")
print(f"Under 20 orgs: {(industry_sizes < 20).sum()} out of {len(industry_sizes)} industries")

INDUSTRY_TYPE breakdown (approved orgs):
industry_type
Technology                  106
E-commerce                  101
Consulting and Marketing     71
dtype: int64

Smallest: 71 orgs

SPECIFIC INDUSTRY breakdown:
industry
mercury_marketing                 45
mercury_software                  45
mercury_retail_wholesale          32
mercury_b2b                       28
mercury_online_retailer_amazon    28
dtype: int64

Median: 12 orgs
Under 20 orgs: 9 out of 15 industries


Decision: Use industry_type. Better sample sizes (71-106 vs mostly <20)

In [32]:
# Treatment assignment based on adoption patterns
print("Treatment logic:")
print("Technology → Feature Credit Card (13% adoption vs 3% others)")
print("E-commerce → Feature Debit Card (49% adoption, consistent)")
print("Consulting → Feature Invoicing (7% vs 1-3% others)")
print("\nControl: Current onboarding (no featured product)")

Treatment logic:
Technology → Feature Credit Card (13% adoption vs 3% others)
E-commerce → Feature Debit Card (49% adoption, consistent)
Consulting → Feature Invoicing (7% vs 1-3% others)

Control: Current onboarding (no featured product)


In [33]:
# Sample size calculation
from scipy import stats

# Current adoption rate (any non-Bank product)
current_adopters = product_users[product_users['product'] != 'Bank Account']['organization_id'].nunique()
total_approved = orgs['got_approved'].sum()
baseline_rate = current_adopters / total_approved
target_rate = baseline_rate * 1.2  # 20% relative lift

print(f'Baseline adoption: {baseline_rate:.1%}')
print(f'Target adoption: {target_rate:.1%}')
print(f'Absolute lift: {target_rate - baseline_rate:.1%}')

# Sample size for 80% power
alpha = 0.05
power = 0.8
effect_size = target_rate - baseline_rate
pooled_p = (baseline_rate + target_rate) / 2

z_alpha = stats.norm.ppf(1 - alpha/2)
z_beta = stats.norm.ppf(power)

n_per_group = ((z_alpha + z_beta)**2 * 2 * pooled_p * (1 - pooled_p)) / effect_size**2
print(f'Need {n_per_group:.0f} per group = {n_per_group * 2:.0f} total approved orgs')

Baseline adoption: 51.8%
Target adoption: 62.2%
Absolute lift: 10.4%
Need 359 per group = 717 total approved orgs


In [34]:
# Duration estimate
print("Duration estimate:")
print("Historical: ~278 approved in 12 months = 23/month")
print(f"Need 717 approved = {717/23:.0f} months")
print("Add implementation buffer: ~6 months total")

Duration estimate:
Historical: ~278 approved in 12 months = 23/month
Need 717 approved = 31 months
Add implementation buffer: ~6 months total


**Primary metric:** 30-day featured product adoption rate

**Randomization:**
- Unit: Organization (at approval)
- Split: 50/50 control/treatment
- Stratify by industry_type

**Analysis plan:**
- Statistical test: Two-sample proportion test
- Significance: α = 0.05
- Guardrails: Approval rate, activation rate, time to deposit

**Decision framework:**
- Strong success (>20% lift) → Full rollout
- Moderate (10-20% lift) → Gradual rollout
- Mixed by industry → Selective rollout
- Weak/neutral → Don't implement
- Guardrail failure → Stop immediately

What about heterogeneous effects? Each industry is different.

Current baselines for featured products:
- Technology/Credit Card: 13.2%
- E-commerce/Debit Card: 48.5% 
- Consulting/Invoicing: 7.0%

Expected HTE scenarios:
- Tech: moderate absolute lift (13% → maybe 18%)
- E-commerce: smaller relative lift due to ceiling (49% → maybe 54%)
- Consulting: potentially large relative lift (7% → maybe 15%)

For analysis: test interaction between treatment and industry_type. Need to correct for multiple testing - 3 comparisons. Watch out for Consulting sample size issues.

Need more comprehensive metrics beyond just adoption rate.

**Primary:**
- Featured product adoption (30-day)
- Time to adoption (survival analysis)

**Secondary:**
- Multi-product adoption - does featuring one help others?
- Revenue impact per org
- Effect on reaching first_active

**Guardrails (must not hurt):**
- Approval rate - currently 56%
- Time to first deposit  
- Overall activation rate - currently 39%

**Diagnostics:**
- Click-through on featured product CTA
- Time spent on product pages
- Support tickets about featured products

**Success criteria:** 10% relative lift minimum for implementation, no guardrail violations, positive on multiple metrics

## Risk Assessment

**E-commerce ceiling effect:** Debit Card already at 49% - limited upside. Maybe feature Credit Card instead?

**Consulting sample size issues:** Only 71 approved orgs, Invoicing at 7% baseline. Might not have power to detect meaningful effects.

**Approval rate cannibalization:** Featuring products too early might overwhelm users. Solution: feature AFTER approval, before first deposit.

**Technical/randomization issues:** Need A/A test first to validate measurement. Daily sample ratio checks.

**Seasonality confounding:** Different industries have different seasonal patterns. E-commerce spikes in Q4, consulting maybe Q1.

## Decision Scenarios

**If all 3 industries show >15% lift:** Full rollout, pretty clear win.

**If only Tech works (>20% lift), others flat:** Rollout to Tech only. Redesign approach for E-commerce and Consulting.

**If 2 out of 3 work:** Rollout to winners. Try different products for the loser industry.

**If statistically significant but <10% lift:** Don't implement, not worth the complexity. Try different messaging or timing.

**If approval rate drops >2%:** Stop immediately, this is killing the business. UX research on why it's overwhelming users.

**If completely null results:** Abandon industry-based approach. Test other segmentation like company size or growth stage.

**Confidence thresholds:** High confidence = n>500 per group + large effect. Medium = n>300 + moderate effect. Low = need follow-up validation experiment.

## Implementation Strategy

Need to be careful about launch. A/A test first to validate measurement - run for 1 week with 10% traffic.

**Early stopping rules:**
- Stop early for strong win: >25% lift with high confidence after 50% sample
- Stop for futility: <2% lift at 75% sample with Bayesian analysis showing <5% chance of success  
- Stop for harm: Any guardrail violation >50% of threshold

**Monitoring:**
- Daily: guardrails, sample ratios, basic health
- Weekly: primary metrics, power updates
- Bi-weekly: deep dive, HTE analysis

**Learning agenda beyond this experiment:**
1. Messaging optimization - which copy works best?
2. Timing - immediate vs delayed vs progressive disclosure?
3. Deeper personalization - specific industry vs company characteristics?
4. Cross-sell effects - does featuring one product help others?

This balances statistical rigor with business needs. Want maximum learning while protecting key metrics."